# Back to the Basics
------------------

## Import ATTACK API Client

In [1]:
from attackcti import attack_client

## Import Extra Libraries

In [2]:
from pandas import *
import json

In [3]:
pandas.__version__

'1.1.2'

## Initialize ATT&CK Client Variable

In [4]:
lift = attack_client()

## **Collect (Enterprise ATT&CK, Pre-ATT&CK & Mobile ATT&CK)**
* I usually collect all the stix object types available from all the ATT&CK Matrices first when I want to analyze ATT&CK's data.
* In this section, we will collect everything from Enterprise ATT&CK, PRE-ATT&CK and Mobile ATT&CK via three functions that query ATT&CK content available in STIX™ 2.0 format via its own public TAXII™ 2.0 server:
  * get_enterprise()
  * get_pre()
  * get_mobile()

**Collect ALL Enterprise ATT&CK (TAXII)**

In [5]:
%time all_enterprise = lift.get_enterprise()

CPU times: user 3.28 s, sys: 127 ms, total: 3.41 s
Wall time: 7.96 s


**Collect ALL PRE-ATT&CK (TAXII)**

In [6]:
%time all_pre = lift.get_pre()

/usr/local/lib/python3.8/site-packages/attackcti/attack_api.py:426: UserWarning: PRE ATT&CK is deprecated. It will be removed in future versions. Consider adjusting your application
  warnings.warn("PRE ATT&CK is deprecated. It will be removed in future versions. Consider adjusting your application")


CPU times: user 110 ms, sys: 7.01 ms, total: 117 ms
Wall time: 392 ms


**Collect ALL Mobile ATT&CK (TAXII)**

In [7]:
%time all_mobile = lift.get_mobile()

CPU times: user 408 ms, sys: 12.7 ms, total: 421 ms
Wall time: 1.05 s


## **Collect ALL (It runs All 3 previous functions and collects all the results)**

The **get_stix_objects()** function returns a dictionary with all the stix object types from all matrices:
* techniques
* mitigations
* groups
* malware
* tools
* relationships

In [8]:
%time all_attack = lift.get_stix_objects()

CPU times: user 3min 58s, sys: 713 ms, total: 3min 59s
Wall time: 4min 5s


In [9]:
type(all_attack)

dict

### Get All Techniques from ATT&CK Results (Locally - STIX format)
* The results of this function shows every single technique across the whole ATT&CK framework without their mitigations information

In [10]:
print("Number of Techniques in ATT&CK")
print(len(all_attack['techniques']))

Number of Techniques in ATT&CK
1062


* By default, the data returned by the available functions in the attackcti library is of type **stix2**
* However, if you want to interact with libraries such as **Pandas**, it needs to be of type **dict**

In [11]:
techniques = []
for t in all_attack['techniques']:
    techniques.append(json.loads(t.serialize()))
df = pandas.json_normalize(techniques)
df.reindex(['created','name', 'x_mitre_data_sources', 'x_mitre_platforms'], axis=1)[0:5]

,created,name,x_mitre_data_sources,x_mitre_platforms
0,2021-04-23T01:04:57.161Z,Code Signing Policy Modification,[Windows Registry: Windows Registry Key Modifi...,"[Windows, macOS]"
1,2021-04-01T16:42:08.735Z,System Location Discovery,"[Instance: Instance Metadata, Process: Process...","[Windows, Linux, macOS, IaaS]"
2,2021-03-31T14:26:00.848Z,Container and Resource Discovery,"[Cluster: Cluster Metadata, Container: Contain...",[Containers]
3,2021-03-31T14:01:52.321Z,Container API,"[Command: Command Execution, File: File Access...",[Containers]
4,2021-03-30T17:54:03.944Z,Build Image on Host,"[Image: Image Creation, Network Traffic: Netwo...",[Containers]


**Showing the schema of Techniques**

This schema covers techniques from Enterprise, PRE and Mobile ATT&CK

In [12]:
list(df)

['external_references',
 'object_marking_refs',
 'created_by_ref',
 'name',
 'description',
 'id',
 'type',
 'kill_chain_phases',
 'modified',
 'created',
 'x_mitre_detection',
 'x_mitre_data_sources',
 'x_mitre_contributors',
 'x_mitre_defense_bypassed',
 'x_mitre_version',
 'x_mitre_is_subtechnique',
 'x_mitre_permissions_required',
 'x_mitre_platforms',
 'x_mitre_remote_support',
 'x_mitre_system_requirements',
 'x_mitre_network_requirements',
 'x_mitre_effective_permissions',
 'x_mitre_impact_type',
 'revoked',
 'x_mitre_deprecated',
 'x_mitre_old_attack_id',
 'x_mitre_difficulty_for_adversary_explanation',
 'x_mitre_difficulty_for_adversary',
 'x_mitre_detectable_by_common_defenses_explanation',
 'x_mitre_detectable_by_common_defenses',
 'x_mitre_tactic_type']

**Showing one technique example**

In [13]:
techniques[0]

{'external_references': [{'source_name': 'mitre-attack',
   'external_id': 'T1553.006',
   'url': 'https://attack.mitre.org/techniques/T1553/006'},
  {'source_name': 'Microsoft DSE June 2017',
   'url': 'https://docs.microsoft.com/en-us/previous-versions/windows/hardware/design/dn653559(v=vs.85)?redirectedfrom=MSDN',
   'description': 'Microsoft. (2017, June 1). Digital Signatures for Kernel Modules on Windows. Retrieved April 22, 2021.'},
  {'source_name': 'Apple Disable SIP',
   'url': 'https://developer.apple.com/documentation/security/disabling_and_enabling_system_integrity_protection',
   'description': 'Apple. (n.d.). Disabling and Enabling System Integrity Protection. Retrieved April 22, 2021.'},
  {'source_name': 'Microsoft Unsigned Driver Apr 2017',
   'url': 'https://docs.microsoft.com/en-us/windows-hardware/drivers/install/installing-an-unsigned-driver-during-development-and-test',
   'description': 'Microsoft. (2017, April 20). Installing an Unsigned Driver during Developme

### Get All Mitigations from ATT&CK Results (Locally)

In [14]:
print("Number of Mitigations in ATT&CK")
print(len(all_attack['mitigations']))

Number of Mitigations in ATT&CK
329


* Remember, we need to transform our results to dictionaries to be able to use them as pandas dataframes

In [15]:
mitigations = []
for t in all_attack['mitigations']:
    mitigations.append(json.loads(t.serialize()))
df = pandas.json_normalize(mitigations)
df[0:4]

,created_by_ref,object_marking_refs,external_references,description,name,id,type,modified,created,x_mitre_version,x_mitre_deprecated,x_mitre_old_attack_id,labels
0,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'source_name': 'mitre-attack', 'external_id'...",This category is used for any applicable mitig...,Pre-compromise,course-of-action--78bb71be-92b4-46de-acd6-5f99...,course-of-action,2020-10-20T19:52:32.439Z,2020-10-19T14:57:58.771Z,1.0,NaN,NaN,NaN
1,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'source_name': 'mitre-attack', 'external_id'...",This category is to associate techniques that ...,Do Not Mitigate,course-of-action--787fb64d-c87b-4ee5-a341-0ef1...,course-of-action,2019-07-23T14:44:24.727Z,2019-07-19T14:58:42.715Z,1.0,NaN,NaN,NaN
2,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'source_name': 'mitre-attack', 'external_id'...",Implement configuration changes to software (o...,Software Configuration,course-of-action--b5dbb4c5-b0b1-40b1-80b6-e9e8...,course-of-action,2020-03-31T13:11:09.471Z,2019-07-19T14:40:23.529Z,1.1,NaN,NaN,NaN
3,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'source_name': 'mitre-attack', 'external_id'...",Take and store data backups from end user syst...,Data Backup,course-of-action--3efe43d1-6f3f-4fcb-ab39-4a73...,course-of-action,2020-03-31T13:11:28.201Z,2019-07-19T14:33:33.543Z,1.1,NaN,NaN,NaN


In [16]:
list(df)

['created_by_ref',
 'object_marking_refs',
 'external_references',
 'description',
 'name',
 'id',
 'type',
 'modified',
 'created',
 'x_mitre_version',
 'x_mitre_deprecated',
 'x_mitre_old_attack_id',
 'labels']

### Get All Groups from ATT&CK Results (Locally)

In [17]:
print("Number of Groups in ATT&CK")
print(len(all_attack['groups']))

Number of Groups in ATT&CK
127


* Once again, we need to transform our results to dictionaries to be able to use them as pandas dataframes

In [18]:
groups = []
for t in all_attack['groups']:
    groups.append(json.loads(t.serialize()))
df = pandas.json_normalize(groups)
df[0:4]

,created_by_ref,object_marking_refs,external_references,description,name,type,id,aliases,modified,created,x_mitre_version,x_mitre_contributors,revoked
0,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'external_id': 'G0130', 'source_name': 'mitr...",[Ajax Security Team](https://attack.mitre.org/...,Ajax Security Team,intrusion-set,intrusion-set--fa19de15-6169-428d-9cd6-3ca3d56...,"[Ajax Security Team, Operation Woolen-Goldfish...",2021-04-22T20:13:14.377Z,2021-04-14T13:17:43.941Z,1.0,NaN,NaN
1,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'external_id': 'G0129', 'source_name': 'mitr...",[Mustang Panda](https://attack.mitre.org/group...,Mustang Panda,intrusion-set,intrusion-set--420ac20b-f2b9-42b8-aa1a-6d4b728...,"[Mustang Panda, TA416, RedDelta, BRONZE PRESID...",2021-04-25T21:15:45.895Z,2021-04-12T15:56:28.861Z,1.0,"[Kyaw Pyiyt Htet, @KyawPyiytHtet]",NaN
2,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'external_id': 'G0128', 'source_name': 'mitr...",[ZIRCONIUM](https://attack.mitre.org/groups/G0...,ZIRCONIUM,intrusion-set,intrusion-set--4283ae19-69c7-4347-a35e-b56f08e...,"[ZIRCONIUM, APT31]",2021-04-20T21:00:44.930Z,2021-03-24T15:48:17.731Z,1.0,NaN,NaN
3,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'external_id': 'G0127', 'source_name': 'mitr...",[TA551](https://attack.mitre.org/groups/G0127)...,TA551,intrusion-set,intrusion-set--94873029-f950-4268-9cfd-5032e15...,"[TA551, GOLD CABIN, Shathak]",2021-03-25T15:33:58.444Z,2021-03-19T21:04:00.692Z,1.0,"[Shuhei Sasada, Cyber Defense Institute, Inc, ...",NaN


**Showing the schema of Groups**

In [19]:
list(df)

['created_by_ref',
 'object_marking_refs',
 'external_references',
 'description',
 'name',
 'type',
 'id',
 'aliases',
 'modified',
 'created',
 'x_mitre_version',
 'x_mitre_contributors',
 'revoked']

**Showing one Groups example**

In [20]:
groups[0]

{'created_by_ref': 'identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5',
 'object_marking_refs': ['marking-definition--fa42a846-8d90-4e51-bc29-71d5b4802168'],
 'external_references': [{'external_id': 'G0130',
   'source_name': 'mitre-attack',
   'url': 'https://attack.mitre.org/groups/G0130'},
  {'source_name': 'Operation Woolen-Goldfish',
   'description': 'Analysis of infrastructure, tools, and modes of operation revealed a potential relationship between Ajax Security Team and the campaign Operation Woolen-Goldfish.(Citation: Check Point Rocket Kitten)(Citation: TrendMicro Operation Woolen Goldfish March 2015)'},
  {'source_name': 'AjaxTM',
   'description': '(Citation: FireEye Operation Saffron Rose 2013)'},
  {'source_name': 'Rocket Kitten',
   'description': 'Analysis of infrastructure, tools, and modes of operation revealed a potential relationship between Ajax Security Team and Rocket Kitten.(Citation: Check Point Rocket Kitten)(Citation: IranThreats Kittens Dec 2017)'},
  {'source_

### Get All Malware objects from ATT&CK Results (Locally)

In [21]:
print("Number of Malware in ATT&CK")
print(len(all_attack['malware']))

Number of Malware in ATT&CK
524


In [22]:
malware = []
for t in all_attack['malware']:
    malware.append(json.loads(t.serialize()))
df = pandas.json_normalize(malware)
df[0:4]

,external_references,object_marking_refs,created_by_ref,description,name,id,type,labels,modified,created,x_mitre_version,x_mitre_aliases,x_mitre_platforms,x_mitre_contributors,revoked,x_mitre_old_attack_id
0,"[{'external_id': 'S0598', 'source_name': 'mitr...",[marking-definition--fa42a846-8d90-4e51-bc29-7...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[P.A.S. Webshell](https://attack.mitre.org/sof...,P.A.S. Webshell,malware--4800d0f9-00aa-47cd-a4d2-92198585b8fd,malware,[malware],2021-04-13T13:10:36.820Z,2021-04-13T12:46:58.579Z,1.0,"[P.A.S. Webshell, Fobushell]","[Linux, Windows]",NaN,NaN,NaN
1,"[{'external_id': 'S0601', 'source_name': 'mitr...",[marking-definition--fa42a846-8d90-4e51-bc29-7...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[Hildegard](https://attack.mitre.org/software/...,Hildegard,malware--40a1b8ec-7295-416c-a6b1-68181d86f120,malware,[malware],2021-04-12T12:51:56.886Z,2021-04-07T18:07:47.604Z,1.0,[Hildegard],"[Linux, Containers, IaaS]",[Center for Threat-Informed Defense (CTID)],NaN,NaN
2,"[{'external_id': 'S0600', 'source_name': 'mitr...",[marking-definition--fa42a846-8d90-4e51-bc29-7...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[Doki](https://attack.mitre.org/software/S0600...,Doki,malware--4f1c389e-a80e-4a3e-9b0e-9be8c91df64f,malware,[malware],2021-04-19T17:45:07.102Z,2021-04-06T15:53:34.722Z,1.0,[Doki],"[Linux, Containers]",[Center for Threat-Informed Defense (CTID)],NaN,NaN
3,"[{'external_id': 'S0599', 'source_name': 'mitr...",[marking-definition--fa42a846-8d90-4e51-bc29-7...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[Kinsing](https://attack.mitre.org/software/S0...,Kinsing,malware--d6e55656-e43f-411f-a7af-45df650471c5,malware,[malware],2021-04-12T12:49:39.027Z,2021-04-06T12:22:23.447Z,1.0,[Kinsing],"[Containers, Linux]",[Center for Threat-Informed Defense (CTID)],NaN,NaN


**Showing the schema of Malware**

In [23]:
list(df)

['external_references',
 'object_marking_refs',
 'created_by_ref',
 'description',
 'name',
 'id',
 'type',
 'labels',
 'modified',
 'created',
 'x_mitre_version',
 'x_mitre_aliases',
 'x_mitre_platforms',
 'x_mitre_contributors',
 'revoked',
 'x_mitre_old_attack_id']

**Showing one Malware example**

In [24]:
malware[0]

{'external_references': [{'external_id': 'S0598',
   'source_name': 'mitre-attack',
   'url': 'https://attack.mitre.org/software/S0598'},
  {'source_name': 'Fobushell',
   'description': '(Citation: NCCIC AR-17-20045 February 2017)'},
  {'source_name': 'ANSSI Sandworm January 2021',
   'url': 'https://www.cert.ssi.gouv.fr/uploads/CERTFR-2021-CTI-005.pdf',
   'description': 'ANSSI. (2021, January 27). SANDWORM INTRUSION SET CAMPAIGN TARGETING CENTREON SYSTEMS. Retrieved March 30, 2021.'},
  {'source_name': 'NCCIC AR-17-20045 February 2017',
   'url': 'https://us-cert.cisa.gov/sites/default/files/publications/AR-17-20045_Enhanced_Analysis_of_GRIZZLY_STEPPE_Activity.pdf',
   'description': 'NCCIC. (2017, February 10). Enhanced Analysis of GRIZZLY STEPPE Activity. Retrieved April 12, 2021.'}],
 'object_marking_refs': ['marking-definition--fa42a846-8d90-4e51-bc29-71d5b4802168'],
 'created_by_ref': 'identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5',
 'description': '[P.A.S. Webshell](https://a

### Get All Tools from ATT&CK Results (Locally)

In [25]:
print("Number of Tools in ATT&CK")
print(len(all_attack['tools']))

Number of Tools in ATT&CK
72


In [26]:
tools = []
for t in all_attack['tools']:
    tools.append(json.loads(t.serialize()))
df = pandas.json_normalize(tools)
df[0:4]

,created,modified,labels,type,id,name,description,created_by_ref,object_marking_refs,external_references,x_mitre_platforms,x_mitre_aliases,x_mitre_version,x_mitre_contributors,x_mitre_old_attack_id
0,2021-03-19T13:11:50.666Z,2021-04-26T22:35:19.315Z,[tool],tool,tool--80c815bb-b24a-4b9c-9d73-ff4c075a278d,Out1,[Out1](https://attack.mitre.org/software/S0594...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'external_id': 'S0594', 'source_name': 'mitr...",[Windows],[Out1],1.0,NaN,NaN
1,2021-03-18T14:57:34.628Z,2021-04-25T23:30:38.375Z,[tool],tool,tool--03c6e0ea-96d3-4b23-9afb-05055663cf4b,RemoteUtilities,[RemoteUtilities](https://attack.mitre.org/sof...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'external_id': 'S0592', 'source_name': 'mitr...",[Windows],[RemoteUtilities],1.0,NaN,NaN
2,2021-03-18T13:39:27.676Z,2021-03-18T14:54:01.053Z,[tool],tool,tool--842976c7-f9c8-41b2-8371-41dc64fbe261,ConnectWise,[ConnectWise](https://attack.mitre.org/softwar...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'external_id': 'S0591', 'source_name': 'mitr...",[Windows],"[ConnectWise, ScreenConnect]",1.0,NaN,NaN
3,2021-03-17T15:26:20.015Z,2021-04-24T20:45:08.323Z,[tool],tool,tool--b63970b7-ddfb-4aee-97b1-80d335e033a8,NBTscan,[NBTscan](https://attack.mitre.org/software/S0...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'external_id': 'S0590', 'source_name': 'mitr...","[Windows, Linux, macOS]",[NBTscan],1.0,"[Daniyal Naeem, BT Security]",NaN


**Showing the schema of Tools**

In [27]:
list(df)

['created',
 'modified',
 'labels',
 'type',
 'id',
 'name',
 'description',
 'created_by_ref',
 'object_marking_refs',
 'external_references',
 'x_mitre_platforms',
 'x_mitre_aliases',
 'x_mitre_version',
 'x_mitre_contributors',
 'x_mitre_old_attack_id']

**Showing one Tool example**

In [28]:
tools[0]

{'created': '2021-03-19T13:11:50.666Z',
 'modified': '2021-04-26T22:35:19.315Z',
 'labels': ['tool'],
 'type': 'tool',
 'id': 'tool--80c815bb-b24a-4b9c-9d73-ff4c075a278d',
 'name': 'Out1',
 'description': '[Out1](https://attack.mitre.org/software/S0594) is a remote access tool written in python and used by [MuddyWater](https://attack.mitre.org/groups/G0069) since at least 2021.(Citation: Trend Micro Muddy Water March 2021)',
 'created_by_ref': 'identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5',
 'object_marking_refs': ['marking-definition--fa42a846-8d90-4e51-bc29-71d5b4802168'],
 'external_references': [{'external_id': 'S0594',
   'source_name': 'mitre-attack',
   'url': 'https://attack.mitre.org/software/S0594'},
  {'source_name': 'Trend Micro Muddy Water March 2021',
   'url': 'https://www.trendmicro.com/en_us/research/21/c/earth-vetala---muddywater-continues-to-target-organizations-in-t.html',
   'description': 'Peretz, A. and Theck, E. (2021, March 5). Earth Vetala – MuddyWater Conti

### Get All Relationships from ATT&CK Results (Locally)

In [29]:
print("Number of Relationships in ATT&CK")
print(len(all_attack['relationships']))

Number of Relationships in ATT&CK
12617


In [30]:
relationships = []
for t in all_attack['relationships']:
    relationships.append(json.loads(t.serialize()))
df = pandas.json_normalize(relationships)
df[0:4]

,created_by_ref,object_marking_refs,external_references,description,id,type,modified,created,source_ref,relationship_type,target_ref
0,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'source_name': 'CISA AppleJeus Feb 2021', 'u...",[AppleJeus](https://attack.mitre.org/software/...,relationship--fbe555c3-5c7b-44e7-a48f-293bdae9...,relationship,2021-04-27T15:46:45.720Z,2021-04-27T15:46:45.720Z,malware--e2d34c63-6f5a-41f5-86a2-e2380f27f858,uses,attack-pattern--ef67e13e-5598-4adc-bdb2-998225...
1,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'source_name': 'CERT-FR PYSA April 2020', 'u...",[Pysa](https://attack.mitre.org/software/S0583...,relationship--fc0b1fb2-f987-4d9c-8470-c40c5179...,relationship,2021-04-27T03:33:35.204Z,2021-04-27T03:33:35.204Z,malware--a19c1197-9414-46e3-986f-0f609ff4a46b,uses,attack-pattern--d63a3fb8-9452-4e9d-a60a-54be68...
2,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,[{'source_name': 'CheckPoint Volatile Cedar Ma...,[Explosive](https://attack.mitre.org/software...,relationship--00038d0e-7fc7-41c3-9055-edb4d87e...,relationship,2021-04-27T01:56:35.810Z,2021-04-27T01:56:35.810Z,malware--6a21e3a4-5ffe-4581-af9a-6a54c7536f44,uses,attack-pattern--707399d6-ab3e-4963-9315-d9d381...
3,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,[{'source_name': 'CheckPoint Volatile Cedar Ma...,[Explosive](https://attack.mitre.org/software/...,relationship--afcc8ef7-cc5e-4e3b-a430-acafb12f...,relationship,2021-04-27T01:56:35.803Z,2021-04-27T01:56:35.803Z,malware--6a21e3a4-5ffe-4581-af9a-6a54c7536f44,uses,attack-pattern--391d824f-0ef1-47a0-b0ee-c59a75...


**Showing the schema of Relationships**

In [31]:
list(df)

['created_by_ref',
 'object_marking_refs',
 'external_references',
 'description',
 'id',
 'type',
 'modified',
 'created',
 'source_ref',
 'relationship_type',
 'target_ref']

**Showing one Relationship example**

In [32]:
relationships[0]

{'created_by_ref': 'identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5',
 'object_marking_refs': ['marking-definition--fa42a846-8d90-4e51-bc29-71d5b4802168'],
 'external_references': [{'source_name': 'CISA AppleJeus Feb 2021',
   'url': 'https://us-cert.cisa.gov/ncas/alerts/aa21-048a',
   'description': 'Cybersecurity and Infrastructure Security Agency. (2021, February 21). AppleJeus: Analysis of North Korea’s Cryptocurrency Malware. Retrieved March 1, 2021.'}],
 'description': "[AppleJeus](https://attack.mitre.org/software/S0584)'s spearphishing links required user interaction to navigate to the malicious website.(Citation: CISA AppleJeus Feb 2021)",
 'id': 'relationship--fbe555c3-5c7b-44e7-a48f-293bdae9de0c',
 'type': 'relationship',
 'modified': '2021-04-27T15:46:45.720Z',
 'created': '2021-04-27T15:46:45.720Z',
 'source_ref': 'malware--e2d34c63-6f5a-41f5-86a2-e2380f27f858',
 'relationship_type': 'uses',
 'target_ref': 'attack-pattern--ef67e13e-5598-4adc-bdb2-998225874fa9'}

### Get All Tactics from ATT&CK Results (Locally)

In [33]:
print("Number of Tactics in ATT&CK")
print(len(all_attack['tactics']))

Number of Tactics in ATT&CK
55


In [34]:
df = pandas.json_normalize(all_attack['tactics'])
df[0:4]

,created_by_ref,object_marking_refs,external_references,name,description,id,type,modified,created,x_mitre_shortname,x_mitre_deprecated
0,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'external_id': 'TA0043', 'source_name': 'mit...",Reconnaissance,The adversary is trying to gather information ...,x-mitre-tactic--daa4cbb1-b4f4-4723-a824-7f1efd...,x-mitre-tactic,2020-10-18T02:04:50.842Z,2020-10-02T14:48:41.809Z,reconnaissance,NaN
1,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'external_id': 'TA0042', 'source_name': 'mit...",Resource Development,The adversary is trying to establish resources...,x-mitre-tactic--d679bca2-e57d-4935-8650-8031c8...,x-mitre-tactic,2020-09-30T16:31:36.322Z,2020-09-30T16:11:59.650Z,resource-development,NaN
2,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'external_id': 'TA0040', 'source_name': 'mit...",Impact,"The adversary is trying to manipulate, interru...",x-mitre-tactic--5569339b-94c2-49ee-afb3-222293...,x-mitre-tactic,2019-07-25T18:42:23.222Z,2019-03-14T18:44:44.639Z,impact,NaN
3,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'external_id': 'TA0007', 'url': 'https://att...",Discovery,The adversary is trying to figure out your env...,x-mitre-tactic--c17c5845-175e-4421-9713-829d05...,x-mitre-tactic,2019-07-19T17:44:13.228Z,2018-10-17T00:14:20.652Z,discovery,NaN


**Showing the schema of Tactics**

In [35]:
list(df)

['created_by_ref',
 'object_marking_refs',
 'external_references',
 'name',
 'description',
 'id',
 'type',
 'modified',
 'created',
 'x_mitre_shortname',
 'x_mitre_deprecated']

### Get All Matrices from ATT&CK Results (Locally)

In [36]:
print("Number of Matrices in ATT&CK")
print(len(all_attack['matrix']))

Number of Matrices in ATT&CK
5


In [37]:
df = pandas.json_normalize(all_attack['matrix'])
df[0:4]

,id,created_by_ref,name,description,external_references,object_marking_refs,type,tactic_refs,modified,created,x_mitre_deprecated
0,x-mitre-matrix--eafc1b4c-5e56-4965-bd4e-66a6a8...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,Enterprise ATT&CK,Below are the tactics and technique representi...,"[{'external_id': 'enterprise-attack', 'source_...",[marking-definition--fa42a846-8d90-4e51-bc29-7...,x-mitre-matrix,[x-mitre-tactic--daa4cbb1-b4f4-4723-a824-7f1ef...,2021-04-29T21:00:19.254Z,2018-10-17T00:14:20.652Z,NaN
1,x-mitre-matrix--2e2c97c3-1908-4e2d-a711-a27d38...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,PRE-ATT&CK,This object is deprecated as its content has b...,"[{'external_id': 'pre-attack', 'source_name': ...",[marking-definition--fa42a846-8d90-4e51-bc29-7...,x-mitre-matrix,[x-mitre-tactic--b2a086f2-d3db-408b-b4d4-e09a1...,2020-10-22T15:43:48.844Z,2018-10-17T00:14:20.652Z,True
2,x-mitre-matrix--5104d5f0-16b7-4aec-8ae3-0a90cd...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,Network-Based Effects,Below are the tactics and techniques represent...,"[{'external_id': 'mobile-attack', 'url': 'http...",[marking-definition--fa42a846-8d90-4e51-bc29-7...,x-mitre-matrix,[x-mitre-tactic--9eb4c21e-4fa8-44c9-b167-dbfc4...,2020-07-02T14:18:17.535Z,2018-10-17T00:14:20.652Z,NaN
3,x-mitre-matrix--a382db5e-d009-4135-b893-0e0ff0...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,Device Access,Below are the tactics and techniques represent...,"[{'external_id': 'mobile-attack', 'url': 'http...",[marking-definition--fa42a846-8d90-4e51-bc29-7...,x-mitre-matrix,[x-mitre-tactic--0a93fd8e-4a83-4c15-8203-db290...,2021-02-09T14:28:47.299Z,2018-10-17T00:14:20.652Z,NaN


**Showing the schema of Tactics**

In [38]:
list(df)

['id',
 'created_by_ref',
 'name',
 'description',
 'external_references',
 'object_marking_refs',
 'type',
 'tactic_refs',
 'modified',
 'created',
 'x_mitre_deprecated']

### Get All Enterprise ATT&CK ONLY from Results (Locally)
* We can also grab the results from each get_* function that we run earlier, and start getting specific stix object types such as techniques, mitigations, groups, malware, tools and relationships on each specific matrix.
* It is important to remember that the stix objec information is being retrieved from the results of the initial three **get_*** functions and not querying the TAXII Server every time we want to get information about a specific stix object type.

**Enterprise Techniques**

In [39]:
print("Number of Techniques in Enterprise ATT&CK")
print(len(all_enterprise['techniques']))

Number of Techniques in Enterprise ATT&CK
692


In [40]:
techniques = []
for t in all_enterprise['techniques']:
    techniques.append(json.loads(t.serialize()))
df = pandas.json_normalize(techniques)
df[0:4]

,external_references,object_marking_refs,created_by_ref,name,description,id,type,kill_chain_phases,modified,created,...,x_mitre_is_subtechnique,x_mitre_permissions_required,x_mitre_platforms,x_mitre_remote_support,x_mitre_system_requirements,x_mitre_network_requirements,x_mitre_effective_permissions,x_mitre_impact_type,revoked,x_mitre_deprecated
0,"[{'source_name': 'mitre-attack', 'external_id'...",[marking-definition--fa42a846-8d90-4e51-bc29-7...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,Code Signing Policy Modification,Adversaries may modify code signing policies t...,attack-pattern--565275d5-fcc3-4b66-b4e7-928e4c...,attack-pattern,"[{'kill_chain_name': 'mitre-attack', 'phase_na...",2021-04-26T15:41:39.155Z,2021-04-23T01:04:57.161Z,...,True,[Administrator],"[Windows, macOS]",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"[{'source_name': 'mitre-attack', 'external_id'...",[marking-definition--fa42a846-8d90-4e51-bc29-7...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,System Location Discovery,\nAdversaries may gather information in an att...,attack-pattern--c877e33f-1df6-40d6-b1e7-ce70f1...,attack-pattern,"[{'kill_chain_name': 'mitre-attack', 'phase_na...",2021-04-20T19:25:49.977Z,2021-04-01T16:42:08.735Z,...,False,[User],"[Windows, Linux, macOS, IaaS]",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"[{'source_name': 'mitre-attack', 'external_id'...",[marking-definition--fa42a846-8d90-4e51-bc29-7...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,Container and Resource Discovery,Adversaries may attempt to discover containers...,attack-pattern--0470e792-32f8-46b0-a351-652bc3...,attack-pattern,"[{'kill_chain_name': 'mitre-attack', 'phase_na...",2021-04-12T18:22:05.737Z,2021-03-31T14:26:00.848Z,...,False,[User],[Containers],NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"[{'source_name': 'mitre-attack', 'external_id'...",[marking-definition--fa42a846-8d90-4e51-bc29-7...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,Container API,Adversaries may gather credentials via APIs wi...,attack-pattern--f8ef3a62-3f44-40a4-abca-761ab2...,attack-pattern,"[{'kill_chain_name': 'mitre-attack', 'phase_na...",2021-04-12T18:20:31.636Z,2021-03-31T14:01:52.321Z,...,True,"[User, Administrator]",[Containers],NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Enterprise Mitigations**

In [41]:
print("Number of Mitigations in Enterprise ATT&CK")
print(len(all_enterprise['mitigations']))

Number of Mitigations in Enterprise ATT&CK
266


In [42]:
mitigations = []
for t in all_enterprise['mitigations']:
    mitigations.append(json.loads(t.serialize()))
df = pandas.json_normalize(mitigations)
df[0:5]

,created_by_ref,object_marking_refs,external_references,description,name,id,type,modified,created,x_mitre_version,x_mitre_deprecated,x_mitre_old_attack_id
0,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'source_name': 'mitre-attack', 'external_id'...",This category is used for any applicable mitig...,Pre-compromise,course-of-action--78bb71be-92b4-46de-acd6-5f99...,course-of-action,2020-10-20T19:52:32.439Z,2020-10-19T14:57:58.771Z,1.0,NaN,NaN
1,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'source_name': 'mitre-attack', 'external_id'...",This category is to associate techniques that ...,Do Not Mitigate,course-of-action--787fb64d-c87b-4ee5-a341-0ef1...,course-of-action,2019-07-23T14:44:24.727Z,2019-07-19T14:58:42.715Z,1.0,NaN,NaN
2,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'source_name': 'mitre-attack', 'external_id'...",Implement configuration changes to software (o...,Software Configuration,course-of-action--b5dbb4c5-b0b1-40b1-80b6-e9e8...,course-of-action,2020-03-31T13:11:09.471Z,2019-07-19T14:40:23.529Z,1.1,NaN,NaN
3,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'source_name': 'mitre-attack', 'external_id'...",Take and store data backups from end user syst...,Data Backup,course-of-action--3efe43d1-6f3f-4fcb-ab39-4a73...,course-of-action,2020-03-31T13:11:28.201Z,2019-07-19T14:33:33.543Z,1.1,NaN,NaN
4,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'source_name': 'mitre-attack', 'external_id'...",Configure Windows User Account Control to miti...,User Account Control,course-of-action--2c2ad92a-d710-41ab-a996-1db1...,course-of-action,2020-03-31T13:49:49.636Z,2019-06-11T17:14:35.170Z,1.1,NaN,NaN


**Enterprise Groups**

In [43]:
print("Number of Groups in Enterprise ATT&CK")
print(len(all_enterprise['groups']))

Number of Groups in Enterprise ATT&CK
124


In [44]:
groups = []
for t in all_enterprise['groups']:
    groups.append(json.loads(t.serialize()))
df = pandas.json_normalize(groups)
df[0:4]

,created_by_ref,object_marking_refs,external_references,description,name,type,id,aliases,modified,created,x_mitre_version,x_mitre_contributors,revoked
0,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'external_id': 'G0130', 'source_name': 'mitr...",[Ajax Security Team](https://attack.mitre.org/...,Ajax Security Team,intrusion-set,intrusion-set--fa19de15-6169-428d-9cd6-3ca3d56...,"[Ajax Security Team, Operation Woolen-Goldfish...",2021-04-22T20:13:14.377Z,2021-04-14T13:17:43.941Z,1.0,NaN,NaN
1,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'external_id': 'G0129', 'source_name': 'mitr...",[Mustang Panda](https://attack.mitre.org/group...,Mustang Panda,intrusion-set,intrusion-set--420ac20b-f2b9-42b8-aa1a-6d4b728...,"[Mustang Panda, TA416, RedDelta, BRONZE PRESID...",2021-04-25T21:15:45.895Z,2021-04-12T15:56:28.861Z,1.0,"[Kyaw Pyiyt Htet, @KyawPyiytHtet]",NaN
2,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'external_id': 'G0128', 'source_name': 'mitr...",[ZIRCONIUM](https://attack.mitre.org/groups/G0...,ZIRCONIUM,intrusion-set,intrusion-set--4283ae19-69c7-4347-a35e-b56f08e...,"[ZIRCONIUM, APT31]",2021-04-20T21:00:44.930Z,2021-03-24T15:48:17.731Z,1.0,NaN,NaN
3,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'external_id': 'G0127', 'source_name': 'mitr...",[TA551](https://attack.mitre.org/groups/G0127)...,TA551,intrusion-set,intrusion-set--94873029-f950-4268-9cfd-5032e15...,"[TA551, GOLD CABIN, Shathak]",2021-03-25T15:33:58.444Z,2021-03-19T21:04:00.692Z,1.0,"[Shuhei Sasada, Cyber Defense Institute, Inc, ...",NaN


**Enterprise Malware**

In [45]:
print("Number of Malware objects in Enterprise ATT&CK")
print(len(all_enterprise['malware']))

Number of Malware objects in Enterprise ATT&CK
424


In [46]:
malware = []
for t in all_enterprise['malware']:
    malware.append(json.loads(t.serialize()))
df = pandas.json_normalize(malware)
df[0:4]

,external_references,object_marking_refs,created_by_ref,description,name,id,type,labels,modified,created,x_mitre_version,x_mitre_aliases,x_mitre_platforms,x_mitre_contributors,revoked
0,"[{'external_id': 'S0598', 'source_name': 'mitr...",[marking-definition--fa42a846-8d90-4e51-bc29-7...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[P.A.S. Webshell](https://attack.mitre.org/sof...,P.A.S. Webshell,malware--4800d0f9-00aa-47cd-a4d2-92198585b8fd,malware,[malware],2021-04-13T13:10:36.820Z,2021-04-13T12:46:58.579Z,1.0,"[P.A.S. Webshell, Fobushell]","[Linux, Windows]",NaN,NaN
1,"[{'external_id': 'S0601', 'source_name': 'mitr...",[marking-definition--fa42a846-8d90-4e51-bc29-7...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[Hildegard](https://attack.mitre.org/software/...,Hildegard,malware--40a1b8ec-7295-416c-a6b1-68181d86f120,malware,[malware],2021-04-12T12:51:56.886Z,2021-04-07T18:07:47.604Z,1.0,[Hildegard],"[Linux, Containers, IaaS]",[Center for Threat-Informed Defense (CTID)],NaN
2,"[{'external_id': 'S0600', 'source_name': 'mitr...",[marking-definition--fa42a846-8d90-4e51-bc29-7...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[Doki](https://attack.mitre.org/software/S0600...,Doki,malware--4f1c389e-a80e-4a3e-9b0e-9be8c91df64f,malware,[malware],2021-04-19T17:45:07.102Z,2021-04-06T15:53:34.722Z,1.0,[Doki],"[Linux, Containers]",[Center for Threat-Informed Defense (CTID)],NaN
3,"[{'external_id': 'S0599', 'source_name': 'mitr...",[marking-definition--fa42a846-8d90-4e51-bc29-7...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[Kinsing](https://attack.mitre.org/software/S0...,Kinsing,malware--d6e55656-e43f-411f-a7af-45df650471c5,malware,[malware],2021-04-12T12:49:39.027Z,2021-04-06T12:22:23.447Z,1.0,[Kinsing],"[Containers, Linux]",[Center for Threat-Informed Defense (CTID)],NaN


**Enterprise Tools**

In [47]:
print("Number of Tools in Enterprise ATT&CK")
print(len(all_enterprise['tools']))

Number of Tools in Enterprise ATT&CK
70


In [48]:
tools = []
for t in all_enterprise['tools']:
    tools.append(json.loads(t.serialize()))
df = pandas.json_normalize(tools)
df[0:4]

,created,modified,labels,type,id,name,description,created_by_ref,object_marking_refs,external_references,x_mitre_platforms,x_mitre_aliases,x_mitre_version,x_mitre_contributors
0,2021-03-19T13:11:50.666Z,2021-04-26T22:35:19.315Z,[tool],tool,tool--80c815bb-b24a-4b9c-9d73-ff4c075a278d,Out1,[Out1](https://attack.mitre.org/software/S0594...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'external_id': 'S0594', 'source_name': 'mitr...",[Windows],[Out1],1.0,NaN
1,2021-03-18T14:57:34.628Z,2021-04-25T23:30:38.375Z,[tool],tool,tool--03c6e0ea-96d3-4b23-9afb-05055663cf4b,RemoteUtilities,[RemoteUtilities](https://attack.mitre.org/sof...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'external_id': 'S0592', 'source_name': 'mitr...",[Windows],[RemoteUtilities],1.0,NaN
2,2021-03-18T13:39:27.676Z,2021-03-18T14:54:01.053Z,[tool],tool,tool--842976c7-f9c8-41b2-8371-41dc64fbe261,ConnectWise,[ConnectWise](https://attack.mitre.org/softwar...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'external_id': 'S0591', 'source_name': 'mitr...",[Windows],"[ConnectWise, ScreenConnect]",1.0,NaN
3,2021-03-17T15:26:20.015Z,2021-04-24T20:45:08.323Z,[tool],tool,tool--b63970b7-ddfb-4aee-97b1-80d335e033a8,NBTscan,[NBTscan](https://attack.mitre.org/software/S0...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'external_id': 'S0590', 'source_name': 'mitr...","[Windows, Linux, macOS]",[NBTscan],1.0,"[Daniyal Naeem, BT Security]"


**Enterprise Relationships**

In [49]:
print("Number of Relationships in Enterprise ATT&CK")
print(len(all_enterprise['relationships']))

Number of Relationships in Enterprise ATT&CK
10986


In [50]:
relations = []
for t in all_enterprise['relationships']:
    relations.append(json.loads(t.serialize()))
df = pandas.json_normalize(relations)
df[0:4]

,created_by_ref,object_marking_refs,external_references,description,id,type,modified,created,source_ref,relationship_type,target_ref
0,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'source_name': 'CISA AppleJeus Feb 2021', 'u...",[AppleJeus](https://attack.mitre.org/software/...,relationship--fbe555c3-5c7b-44e7-a48f-293bdae9...,relationship,2021-04-27T15:46:45.720Z,2021-04-27T15:46:45.720Z,malware--e2d34c63-6f5a-41f5-86a2-e2380f27f858,uses,attack-pattern--ef67e13e-5598-4adc-bdb2-998225...
1,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'source_name': 'CERT-FR PYSA April 2020', 'u...",[Pysa](https://attack.mitre.org/software/S0583...,relationship--fc0b1fb2-f987-4d9c-8470-c40c5179...,relationship,2021-04-27T03:33:35.204Z,2021-04-27T03:33:35.204Z,malware--a19c1197-9414-46e3-986f-0f609ff4a46b,uses,attack-pattern--d63a3fb8-9452-4e9d-a60a-54be68...
2,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,[{'source_name': 'CheckPoint Volatile Cedar Ma...,[Explosive](https://attack.mitre.org/software...,relationship--00038d0e-7fc7-41c3-9055-edb4d87e...,relationship,2021-04-27T01:56:35.810Z,2021-04-27T01:56:35.810Z,malware--6a21e3a4-5ffe-4581-af9a-6a54c7536f44,uses,attack-pattern--707399d6-ab3e-4963-9315-d9d381...
3,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,[{'source_name': 'CheckPoint Volatile Cedar Ma...,[Explosive](https://attack.mitre.org/software/...,relationship--afcc8ef7-cc5e-4e3b-a430-acafb12f...,relationship,2021-04-27T01:56:35.803Z,2021-04-27T01:56:35.803Z,malware--6a21e3a4-5ffe-4581-af9a-6a54c7536f44,uses,attack-pattern--391d824f-0ef1-47a0-b0ee-c59a75...


### Get All Mobile ATT&CK ONLY from Results (Locally)

**Mobile Techniques**

In [51]:
print("Number of Techniques in Mobile ATT&CK")
print(len(all_mobile['techniques']))

Number of Techniques in Mobile ATT&CK
107


In [52]:
techniques = []
for t in all_mobile['techniques']:
    techniques.append(json.loads(t.serialize()))
df = pandas.json_normalize(techniques)
df[0:4]

,id,description,name,created_by_ref,object_marking_refs,external_references,type,kill_chain_phases,modified,created,x_mitre_platforms,x_mitre_detection,x_mitre_tactic_type,x_mitre_is_subtechnique,x_mitre_version,x_mitre_contributors,x_mitre_old_attack_id,revoked,x_mitre_deprecated
0,attack-pattern--e083305c-49e7-4c87-aae8-968921...,Adversaries may use built-in command-line inte...,Command-Line Interface,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'source_name': 'mitre-mobile-attack', 'exter...",attack-pattern,"[{'kill_chain_name': 'mitre-mobile-attack', 'p...",2020-12-17T17:31:52.802Z,2020-12-16T20:16:07.673Z,"[Android, iOS]",Command-Line Interface execution can be diffic...,[Post-Adversary Device Access],False,1.0,NaN,NaN,NaN,NaN
1,attack-pattern--5ca3c7ec-55b2-4587-9376-cf6c96...,Adversaries may use a compromised device as a ...,Proxy Through Victim,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'source_name': 'mitre-mobile-attack', 'exter...",attack-pattern,"[{'kill_chain_name': 'mitre-mobile-attack', 'p...",2020-12-04T20:30:31.513Z,2020-11-30T14:26:07.728Z,[Android],Enterprises may be able to detect anomalous tr...,[Post-Adversary Device Access],False,1.0,NaN,NaN,NaN,NaN
2,attack-pattern--00290ac5-551e-44aa-bbd8-c4b913...,Adversaries may abuse task scheduling function...,Scheduled Task/Job,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'source_name': 'mitre-mobile-attack', 'exter...",attack-pattern,"[{'kill_chain_name': 'mitre-mobile-attack', 'p...",2020-11-04T19:45:38.144Z,2020-11-04T16:43:31.619Z,"[Android, iOS]",Scheduling tasks/jobs can be difficult to dete...,[Post-Adversary Device Access],False,1.0,"[Lorin Wu, Trend Micro]",NaN,NaN,NaN
3,attack-pattern--b327a9c0-e709-495c-aa6e-00b042...,"Adversaries may delete, alter, or send SMS mes...",SMS Control,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'source_name': 'mitre-mobile-attack', 'exter...",attack-pattern,"[{'kill_chain_name': 'mitre-mobile-attack', 'p...",2020-10-22T17:04:15.578Z,2020-09-11T15:14:33.730Z,[Android],Users can view the default SMS handler in syst...,[Post-Adversary Device Access],False,1.0,NaN,NaN,NaN,NaN


**Mobile Mitigations**

In [53]:
print("Number of Mitigations in Mobile ATT&CK")
print(len(all_mobile['mitigations']))

Number of Mitigations in Mobile ATT&CK
13


In [54]:
mitigations = []
for t in all_mobile['mitigations']:
    mitigations.append(json.loads(t.serialize()))
df = pandas.json_normalize(mitigations)
df[0:4]

,created_by_ref,object_marking_refs,external_references,name,description,id,type,modified,created,x_mitre_version,x_mitre_old_attack_id
0,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'source_name': 'mitre-attack', 'external_id'...",User Guidance,Describes any guidance or training given to us...,course-of-action--653492e3-27be-4a0e-b08c-938d...,course-of-action,2019-10-18T15:51:48.318Z,2019-10-18T12:53:03.508Z,1.0,NaN
1,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'source_name': 'mitre-attack', 'external_id'...",Security Updates,Install security updates in response to discov...,course-of-action--bcecd036-f40e-4916-9f8e-fd0c...,course-of-action,2019-10-18T14:56:15.631Z,2019-10-18T12:51:36.488Z,1.0,NaN
2,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'source_name': 'mitre-attack', 'external_id'...",Attestation,Enable remote attestation capabilities when av...,course-of-action--ff4821f6-5afb-481b-8c0f-26c2...,course-of-action,2019-10-18T14:52:53.019Z,2019-10-18T12:50:35.335Z,1.0,NaN
3,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'source_name': 'mitre-attack', 'external_id'...",Application Vetting,Enterprises can vet applications for exploitab...,course-of-action--1553b156-6767-47f7-9eb4-2a69...,course-of-action,2021-02-18T16:14:17.809Z,2019-10-18T12:49:58.924Z,1.0,NaN


**Mobile Groups**

In [55]:
print("Number of Groups in Mobile ATT&CK")
print(len(all_mobile['groups']))

Number of Groups in Mobile ATT&CK
5


In [56]:
groups = []
for t in all_mobile['groups']:
    groups.append(json.loads(t.serialize()))
df = pandas.json_normalize(groups)
df[0:4]

,created,modified,aliases,created_by_ref,object_marking_refs,external_references,description,name,type,id,x_mitre_version,x_mitre_contributors
0,2020-06-25T17:16:39.168Z,2021-04-26T14:37:33.234Z,"[Windshift, Bahamut]",identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'external_id': 'G0112', 'source_name': 'mitr...",[Windshift](https://attack.mitre.org/groups/G0...,Windshift,intrusion-set,intrusion-set--afec6dc3-a18e-4b62-b1a4-5510e1a...,1.1,NaN
1,2020-01-27T16:55:39.688Z,2020-03-26T20:58:44.722Z,[Bouncing Golf],identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'external_id': 'G0097', 'source_name': 'mitr...",[Bouncing Golf](https://attack.mitre.org/group...,Bouncing Golf,intrusion-set,intrusion-set--049cef3b-22d5-4be6-b50c-9839c7a...,1.0,NaN
2,2018-10-17T00:14:20.652Z,2020-06-03T20:22:40.401Z,[Dark Caracal],identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'source_name': 'mitre-attack', 'url': 'https...",[Dark Caracal](https://attack.mitre.org/groups...,Dark Caracal,intrusion-set,intrusion-set--8a831aaa-f3e0-47a3-bed8-a9ced74...,1.2,NaN
3,2017-05-31T21:32:04.588Z,2021-04-13T13:13:09.511Z,"[Sandworm Team, ELECTRUM, Telebots, IRON VIKIN...",identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'source_name': 'mitre-attack', 'url': 'https...",[Sandworm Team](https://attack.mitre.org/group...,Sandworm Team,intrusion-set,intrusion-set--381fcf73-60f6-4ab2-9991-6af3cbc...,2.0,NaN


**Mobile Malware**

In [57]:
print("Number of Malware in Mobile ATT&CK")
print(len(all_mobile['malware']))

Number of Malware in Mobile ATT&CK
91


In [58]:
malware = []
for t in all_mobile['malware']:
    malware.append(json.loads(t.serialize()))
df = pandas.json_normalize(malware)
df[0:4]

,external_references,object_marking_refs,created_by_ref,description,name,id,type,labels,modified,created,x_mitre_version,x_mitre_aliases,x_mitre_platforms,x_mitre_contributors,x_mitre_old_attack_id
0,"[{'external_id': 'S0602', 'source_name': 'mitr...",[marking-definition--fa42a846-8d90-4e51-bc29-7...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[Circles](https://attack.mitre.org/software/S0...,Circles,malware--c6a07c89-a24c-4c7e-9e3e-6153cc595e24,malware,[malware],2021-04-26T15:33:55.798Z,2021-04-26T15:33:55.798Z,1.0,[Circles],NaN,NaN,NaN
1,"[{'external_id': 'S0577', 'source_name': 'mitr...",[marking-definition--fa42a846-8d90-4e51-bc29-7...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[FrozenCell](https://attack.mitre.org/software...,FrozenCell,malware--96ea1e13-d50f-45f1-b0cf-4ac9bc5a2d62,malware,[malware],2021-04-19T14:07:24.519Z,2021-02-17T20:43:52.033Z,1.0,[FrozenCell],[Android],NaN,NaN
2,"[{'external_id': 'S0558', 'source_name': 'mitr...",[marking-definition--fa42a846-8d90-4e51-bc29-7...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[Tiktok Pro](https://attack.mitre.org/software...,Tiktok Pro,malware--c6abcaf8-1765-41f8-9fe5-03d42fd0f6c0,malware,[malware],2021-04-19T16:30:16.930Z,2021-01-05T20:16:19.968Z,1.0,[Tiktok Pro],[Android],NaN,NaN
3,"[{'external_id': 'S0555', 'source_name': 'mitr...",[marking-definition--fa42a846-8d90-4e51-bc29-7...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[CHEMISTGAMES](https://attack.mitre.org/softwa...,CHEMISTGAMES,malware--a0d774e4-bafc-4292-8651-3ec899391341,malware,[malware],2021-03-25T16:42:05.526Z,2020-12-31T18:25:04.779Z,1.0,[CHEMISTGAMES],[Android],NaN,NaN


**Mobile Tools**

In [59]:
print("Number of Tools in Mobile ATT&CK")
print(len(all_mobile['tools']))

Number of Tools in Mobile ATT&CK
2


In [60]:
tools = []
for t in all_mobile['tools']:
    tools.append(json.loads(t.serialize()))
df = pandas.json_normalize(tools)
df[0:4]

,external_references,object_marking_refs,created_by_ref,description,name,id,type,labels,modified,created,x_mitre_version,x_mitre_aliases,x_mitre_platforms,x_mitre_contributors,x_mitre_old_attack_id
0,"[{'external_id': 'S0408', 'source_name': 'mitr...",[marking-definition--fa42a846-8d90-4e51-bc29-7...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[FlexiSpy](https://attack.mitre.org/software/S...,FlexiSpy,tool--1622fd3d-fcfc-4d02-ac49-f2d786f79b81,tool,[tool],2019-10-14T18:08:28.349Z,2019-09-04T15:38:56.070Z,1.0,[FlexiSpy],[Android],"[Emily Ratliff, IBM]",NaN
1,"[{'source_name': 'mitre-mobile-attack', 'url':...",[marking-definition--fa42a846-8d90-4e51-bc29-7...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[Xbot](https://attack.mitre.org/software/S0298...,Xbot,tool--da21929e-40c0-443d-bdf4-6b60d15448b4,tool,[tool],2018-12-11T20:40:31.461Z,2017-10-25T14:48:48.609Z,1.1,[Xbot],[Android],NaN,MOB-S0014


**Mobile Relationships**

In [61]:
print("Number of Relationships in Mobile ATT&CK")
print(len(all_mobile['relationships']))

Number of Relationships in Mobile ATT&CK
1050


In [62]:
relations = []
for t in all_mobile['relationships']:
    relations.append(json.loads(t.serialize()))
df = pandas.json_normalize(relations)
df[0:4]

,created_by_ref,object_marking_refs,external_references,description,id,type,modified,created,source_ref,relationship_type,target_ref
0,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'source_name': 'CitizenLab Circles', 'url': ...",[Circles](https://attack.mitre.org/software/S0...,relationship--718a612e-50c5-40ab-9081-b88cefea...,relationship,2021-04-26T15:33:55.905Z,2021-04-26T15:33:55.905Z,malware--c6a07c89-a24c-4c7e-9e3e-6153cc595e24,uses,attack-pattern--52651225-0b3a-482d-aa7e-10618f...
1,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'source_name': 'CitizenLab Circles', 'url': ...",[Circles](https://attack.mitre.org/software/S0...,relationship--9de24ec5-63bc-4520-9f81-d1b93a31...,relationship,2021-04-26T15:33:55.897Z,2021-04-26T15:33:55.897Z,malware--c6a07c89-a24c-4c7e-9e3e-6153cc595e24,uses,attack-pattern--fb3fa94a-3aee-4ab0-b7e7-abdf0a...
2,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'source_name': 'Lookout Uyghur Campaign', 'u...",[DoubleAgent](https://attack.mitre.org/softwar...,relationship--64ddcf35-dbf0-4b9f-bf07-1e0bde8b...,relationship,2021-04-19T17:05:42.574Z,2021-04-19T17:05:42.574Z,malware--3d6c4389-3489-40a3-beda-c56e650b6f68,uses,attack-pattern--e1c912a9-e305-434b-9172-8a6ce3...
3,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'source_name': 'Lookout Uyghur Campaign', 'u...",[SilkBean](https://attack.mitre.org/software/...,relationship--2f8b5252-551c-4a0d-8e72-8da40507...,relationship,2021-04-19T14:29:46.530Z,2021-04-19T14:29:46.530Z,malware--ddbe5657-e21e-4a89-8221-2f1362d397ec,uses,attack-pattern--b327a9c0-e709-495c-aa6e-00b042...


## **Get STIX Object Types Directly from TAXII Server (Enterprise ATT&CK, Pre-ATT&CK & Mobile ATT&CK)**
* In this section, we will query the ATT&CK TAXII Server directly in order to collect specific stix object types such as techniques, mitigations, groups, malware, tools and relationships from the Enterprise, PRE and Mobile Matrices.
* There is no need to get all the stix objects available per each matrix unlike the first section of this notebook.

### Get All Enterprise Techniques ONLY (TAXII)

In [63]:
print("Number of Techniques in Enterprise ATT&CK")
techniques = lift.get_enterprise_techniques()
print(len(techniques))

Number of Techniques in Enterprise ATT&CK
692


In [64]:
techniques_list = []
for t in techniques:
    techniques_list.append(json.loads(t.serialize()))
df = pandas.json_normalize(techniques_list)
df[0:4]

,external_references,object_marking_refs,created_by_ref,name,description,id,type,kill_chain_phases,modified,created,...,x_mitre_is_subtechnique,x_mitre_permissions_required,x_mitre_platforms,x_mitre_remote_support,x_mitre_system_requirements,x_mitre_network_requirements,x_mitre_effective_permissions,x_mitre_impact_type,revoked,x_mitre_deprecated
0,"[{'source_name': 'mitre-attack', 'external_id'...",[marking-definition--fa42a846-8d90-4e51-bc29-7...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,Code Signing Policy Modification,Adversaries may modify code signing policies t...,attack-pattern--565275d5-fcc3-4b66-b4e7-928e4c...,attack-pattern,"[{'kill_chain_name': 'mitre-attack', 'phase_na...",2021-04-26T15:41:39.155Z,2021-04-23T01:04:57.161Z,...,True,[Administrator],"[Windows, macOS]",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"[{'source_name': 'mitre-attack', 'external_id'...",[marking-definition--fa42a846-8d90-4e51-bc29-7...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,System Location Discovery,\nAdversaries may gather information in an att...,attack-pattern--c877e33f-1df6-40d6-b1e7-ce70f1...,attack-pattern,"[{'kill_chain_name': 'mitre-attack', 'phase_na...",2021-04-20T19:25:49.977Z,2021-04-01T16:42:08.735Z,...,False,[User],"[Windows, Linux, macOS, IaaS]",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"[{'source_name': 'mitre-attack', 'external_id'...",[marking-definition--fa42a846-8d90-4e51-bc29-7...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,Container and Resource Discovery,Adversaries may attempt to discover containers...,attack-pattern--0470e792-32f8-46b0-a351-652bc3...,attack-pattern,"[{'kill_chain_name': 'mitre-attack', 'phase_na...",2021-04-12T18:22:05.737Z,2021-03-31T14:26:00.848Z,...,False,[User],[Containers],NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"[{'source_name': 'mitre-attack', 'external_id'...",[marking-definition--fa42a846-8d90-4e51-bc29-7...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,Container API,Adversaries may gather credentials via APIs wi...,attack-pattern--f8ef3a62-3f44-40a4-abca-761ab2...,attack-pattern,"[{'kill_chain_name': 'mitre-attack', 'phase_na...",2021-04-12T18:20:31.636Z,2021-03-31T14:01:52.321Z,...,True,"[User, Administrator]",[Containers],NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Get All PRE Techniques ONLY (TAXII)

In [65]:
print("Number of Techniques in PRE-ATT&CK")
techniques = lift.get_pre_techniques()
print(len(techniques))

Number of Techniques in PRE-ATT&CK


/usr/local/lib/python3.8/site-packages/attackcti/attack_api.py:455: UserWarning: PRE ATT&CK is deprecated. It will be removed in future versions. Consider adjusting your application
  warnings.warn("PRE ATT&CK is deprecated. It will be removed in future versions. Consider adjusting your application")


174


In [66]:
techniques_list = []
for t in techniques:
    techniques_list.append(json.loads(t.serialize()))
df = pandas.json_normalize(techniques_list)
df[0:4]

,id,created_by_ref,name,description,external_references,object_marking_refs,type,kill_chain_phases,modified,created,x_mitre_is_subtechnique,x_mitre_old_attack_id,x_mitre_version,x_mitre_difficulty_for_adversary_explanation,x_mitre_difficulty_for_adversary,x_mitre_detectable_by_common_defenses_explanation,x_mitre_detectable_by_common_defenses,x_mitre_deprecated
0,attack-pattern--b182f29c-2505-4b32-a000-0440ef...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,Spearphishing for Information,This object is deprecated as its content has b...,"[{'source_name': 'mitre-pre-attack', 'url': 'h...",[marking-definition--fa42a846-8d90-4e51-bc29-7...,attack-pattern,"[{'kill_chain_name': 'mitre-pre-attack', 'phas...",2020-10-26T13:42:49.342Z,2018-04-18T17:59:24.739Z,False,PRE-T1174,1.0,"Sending emails is trivial, and, over time, an ...",Yes,"Depending on the specific method of phishing, ...",Partial,True
1,attack-pattern--795c1a92-3a26-453e-b99a-6a566a...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,Acquire and/or use 3rd party infrastructure se...,This object is deprecated as its content has b...,"[{'source_name': 'mitre-pre-attack', 'url': 'h...",[marking-definition--fa42a846-8d90-4e51-bc29-7...,attack-pattern,"[{'kill_chain_name': 'mitre-pre-attack', 'phas...",2020-10-26T13:42:49.342Z,2017-12-14T16:46:06.044Z,NaN,PRE-T1106,1.0,Wide variety of cloud/VPS/hosting/compute/stor...,Yes,Hard to differentiate from standard business o...,No,True
2,attack-pattern--03f4a766-7a21-4b5e-9ccf-e0cf42...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,Acquire or compromise 3rd party signing certif...,This object is deprecated as its content has b...,"[{'source_name': 'mitre-pre-attack', 'url': 'h...",[marking-definition--fa42a846-8d90-4e51-bc29-7...,attack-pattern,"[{'kill_chain_name': 'mitre-pre-attack', 'phas...",2020-10-26T13:42:49.342Z,2017-12-14T16:46:06.044Z,NaN,PRE-T1109,1.0,It is trivial to purchase code signing certifi...,No,Defender will not know what certificates an ad...,No,True
3,attack-pattern--773950e1-090c-488b-a480-9ff236...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,Analyze data collected,This object is deprecated as its content has b...,"[{'source_name': 'mitre-pre-attack', 'url': 'h...",[marking-definition--fa42a846-8d90-4e51-bc29-7...,attack-pattern,"[{'kill_chain_name': 'mitre-pre-attack', 'phas...",2020-10-26T13:42:49.342Z,2017-12-14T16:46:06.044Z,NaN,PRE-T1064,1.0,Many of the common tools highlight these weakn...,Yes,This can be done offline after the data has be...,No,True


### Get All Mobile Techniques ONLY (TAXII)

In [67]:
print("Number of Techniques in Mobile ATT&CK")
techniques = lift.get_mobile_techniques()
print(len(techniques))

Number of Techniques in Mobile ATT&CK
107


In [68]:
techniques_list = []
for t in techniques:
    techniques_list.append(json.loads(t.serialize()))
df = pandas.json_normalize(techniques_list)
df[0:4]

,id,description,name,created_by_ref,object_marking_refs,external_references,type,kill_chain_phases,modified,created,x_mitre_platforms,x_mitre_detection,x_mitre_tactic_type,x_mitre_is_subtechnique,x_mitre_version,x_mitre_contributors,x_mitre_old_attack_id,revoked,x_mitre_deprecated
0,attack-pattern--e083305c-49e7-4c87-aae8-968921...,Adversaries may use built-in command-line inte...,Command-Line Interface,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'source_name': 'mitre-mobile-attack', 'exter...",attack-pattern,"[{'kill_chain_name': 'mitre-mobile-attack', 'p...",2020-12-17T17:31:52.802Z,2020-12-16T20:16:07.673Z,"[Android, iOS]",Command-Line Interface execution can be diffic...,[Post-Adversary Device Access],False,1.0,NaN,NaN,NaN,NaN
1,attack-pattern--5ca3c7ec-55b2-4587-9376-cf6c96...,Adversaries may use a compromised device as a ...,Proxy Through Victim,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'source_name': 'mitre-mobile-attack', 'exter...",attack-pattern,"[{'kill_chain_name': 'mitre-mobile-attack', 'p...",2020-12-04T20:30:31.513Z,2020-11-30T14:26:07.728Z,[Android],Enterprises may be able to detect anomalous tr...,[Post-Adversary Device Access],False,1.0,NaN,NaN,NaN,NaN
2,attack-pattern--00290ac5-551e-44aa-bbd8-c4b913...,Adversaries may abuse task scheduling function...,Scheduled Task/Job,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'source_name': 'mitre-mobile-attack', 'exter...",attack-pattern,"[{'kill_chain_name': 'mitre-mobile-attack', 'p...",2020-11-04T19:45:38.144Z,2020-11-04T16:43:31.619Z,"[Android, iOS]",Scheduling tasks/jobs can be difficult to dete...,[Post-Adversary Device Access],False,1.0,"[Lorin Wu, Trend Micro]",NaN,NaN,NaN
3,attack-pattern--b327a9c0-e709-495c-aa6e-00b042...,"Adversaries may delete, alter, or send SMS mes...",SMS Control,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'source_name': 'mitre-mobile-attack', 'exter...",attack-pattern,"[{'kill_chain_name': 'mitre-mobile-attack', 'p...",2020-10-22T17:04:15.578Z,2020-09-11T15:14:33.730Z,[Android],Users can view the default SMS handler in syst...,[Post-Adversary Device Access],False,1.0,NaN,NaN,NaN,NaN


### Get All Techniques (TAXII)
* The results of this function shows every single technique across the whole ATT&CK framework without their mitigations information
* Mitigations information has its own stix object type (Mitigation) that needs to be correlated with the help of relationship properties
* There is a function already created in this library named **get_techniques_with_mitigations()** that allows you to get a more complete view of techniques

In [69]:
print("Number of Techniques in ATT&CK")
techniques = lift.get_techniques()
print(len(techniques))

Number of Techniques in ATT&CK
1062


In [70]:
techniques_list = []
for t in techniques:
    techniques_list.append(json.loads(t.serialize()))
df = pandas.json_normalize(techniques_list)
df[0:4]

,external_references,object_marking_refs,created_by_ref,name,description,id,type,kill_chain_phases,modified,created,...,x_mitre_effective_permissions,x_mitre_impact_type,revoked,x_mitre_deprecated,x_mitre_old_attack_id,x_mitre_difficulty_for_adversary_explanation,x_mitre_difficulty_for_adversary,x_mitre_detectable_by_common_defenses_explanation,x_mitre_detectable_by_common_defenses,x_mitre_tactic_type
0,"[{'source_name': 'mitre-attack', 'external_id'...",[marking-definition--fa42a846-8d90-4e51-bc29-7...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,Code Signing Policy Modification,Adversaries may modify code signing policies t...,attack-pattern--565275d5-fcc3-4b66-b4e7-928e4c...,attack-pattern,"[{'kill_chain_name': 'mitre-attack', 'phase_na...",2021-04-26T15:41:39.155Z,2021-04-23T01:04:57.161Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"[{'source_name': 'mitre-attack', 'external_id'...",[marking-definition--fa42a846-8d90-4e51-bc29-7...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,System Location Discovery,\nAdversaries may gather information in an att...,attack-pattern--c877e33f-1df6-40d6-b1e7-ce70f1...,attack-pattern,"[{'kill_chain_name': 'mitre-attack', 'phase_na...",2021-04-20T19:25:49.977Z,2021-04-01T16:42:08.735Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"[{'source_name': 'mitre-attack', 'external_id'...",[marking-definition--fa42a846-8d90-4e51-bc29-7...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,Container and Resource Discovery,Adversaries may attempt to discover containers...,attack-pattern--0470e792-32f8-46b0-a351-652bc3...,attack-pattern,"[{'kill_chain_name': 'mitre-attack', 'phase_na...",2021-04-12T18:22:05.737Z,2021-03-31T14:26:00.848Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"[{'source_name': 'mitre-attack', 'external_id'...",[marking-definition--fa42a846-8d90-4e51-bc29-7...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,Container API,Adversaries may gather credentials via APIs wi...,attack-pattern--f8ef3a62-3f44-40a4-abca-761ab2...,attack-pattern,"[{'kill_chain_name': 'mitre-attack', 'phase_na...",2021-04-12T18:20:31.636Z,2021-03-31T14:01:52.321Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Get All Enterprise Mitigations ONLY (TAXII)

In [71]:
print("Number of Mitigations in Enterprise ATT&CK")
mitigations = lift.get_enterprise_mitigations()
print(len(mitigations))

Number of Mitigations in Enterprise ATT&CK
266


In [72]:
mitigations_list = []
for t in mitigations:
    mitigations_list.append(json.loads(t.serialize()))
df = pandas.json_normalize(mitigations_list)
df[0:4]

,created_by_ref,object_marking_refs,external_references,description,name,id,type,modified,created,x_mitre_version,x_mitre_deprecated,x_mitre_old_attack_id
0,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'source_name': 'mitre-attack', 'external_id'...",This category is used for any applicable mitig...,Pre-compromise,course-of-action--78bb71be-92b4-46de-acd6-5f99...,course-of-action,2020-10-20T19:52:32.439Z,2020-10-19T14:57:58.771Z,1.0,NaN,NaN
1,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'source_name': 'mitre-attack', 'external_id'...",This category is to associate techniques that ...,Do Not Mitigate,course-of-action--787fb64d-c87b-4ee5-a341-0ef1...,course-of-action,2019-07-23T14:44:24.727Z,2019-07-19T14:58:42.715Z,1.0,NaN,NaN
2,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'source_name': 'mitre-attack', 'external_id'...",Implement configuration changes to software (o...,Software Configuration,course-of-action--b5dbb4c5-b0b1-40b1-80b6-e9e8...,course-of-action,2020-03-31T13:11:09.471Z,2019-07-19T14:40:23.529Z,1.1,NaN,NaN
3,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'source_name': 'mitre-attack', 'external_id'...",Take and store data backups from end user syst...,Data Backup,course-of-action--3efe43d1-6f3f-4fcb-ab39-4a73...,course-of-action,2020-03-31T13:11:28.201Z,2019-07-19T14:33:33.543Z,1.1,NaN,NaN


### Get All Mobile Mitigations ONLY (TAXII)

In [73]:
print("Number of Mitigations in Mobile ATT&CK")
mitigations = lift.get_mobile_mitigations()
print(len(mitigations))

Number of Mitigations in Mobile ATT&CK
13


In [74]:
mitigations_list = []
for t in mitigations:
    mitigations_list.append(json.loads(t.serialize()))
df = pandas.json_normalize(mitigations_list)
df[0:4]

,created_by_ref,object_marking_refs,external_references,name,description,id,type,modified,created,x_mitre_version,x_mitre_old_attack_id
0,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'source_name': 'mitre-attack', 'external_id'...",User Guidance,Describes any guidance or training given to us...,course-of-action--653492e3-27be-4a0e-b08c-938d...,course-of-action,2019-10-18T15:51:48.318Z,2019-10-18T12:53:03.508Z,1.0,NaN
1,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'source_name': 'mitre-attack', 'external_id'...",Security Updates,Install security updates in response to discov...,course-of-action--bcecd036-f40e-4916-9f8e-fd0c...,course-of-action,2019-10-18T14:56:15.631Z,2019-10-18T12:51:36.488Z,1.0,NaN
2,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'source_name': 'mitre-attack', 'external_id'...",Attestation,Enable remote attestation capabilities when av...,course-of-action--ff4821f6-5afb-481b-8c0f-26c2...,course-of-action,2019-10-18T14:52:53.019Z,2019-10-18T12:50:35.335Z,1.0,NaN
3,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'source_name': 'mitre-attack', 'external_id'...",Application Vetting,Enterprises can vet applications for exploitab...,course-of-action--1553b156-6767-47f7-9eb4-2a69...,course-of-action,2021-02-18T16:14:17.809Z,2019-10-18T12:49:58.924Z,1.0,NaN


### Get All Mitigations (TAXII)

In [75]:
print("Number of Mitigations in ATT&CK")
mitigations = lift.get_mitigations()
print(len(mitigations))

Number of Mitigations in ATT&CK
329


In [76]:
mitigations_list = []
for t in mitigations:
    mitigations_list.append(json.loads(t.serialize()))
df = pandas.json_normalize(mitigations_list)
df[0:4]

,created_by_ref,object_marking_refs,external_references,description,name,id,type,modified,created,x_mitre_version,x_mitre_deprecated,x_mitre_old_attack_id,labels
0,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'source_name': 'mitre-attack', 'external_id'...",This category is used for any applicable mitig...,Pre-compromise,course-of-action--78bb71be-92b4-46de-acd6-5f99...,course-of-action,2020-10-20T19:52:32.439Z,2020-10-19T14:57:58.771Z,1.0,NaN,NaN,NaN
1,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'source_name': 'mitre-attack', 'external_id'...",This category is to associate techniques that ...,Do Not Mitigate,course-of-action--787fb64d-c87b-4ee5-a341-0ef1...,course-of-action,2019-07-23T14:44:24.727Z,2019-07-19T14:58:42.715Z,1.0,NaN,NaN,NaN
2,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'source_name': 'mitre-attack', 'external_id'...",Implement configuration changes to software (o...,Software Configuration,course-of-action--b5dbb4c5-b0b1-40b1-80b6-e9e8...,course-of-action,2020-03-31T13:11:09.471Z,2019-07-19T14:40:23.529Z,1.1,NaN,NaN,NaN
3,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'source_name': 'mitre-attack', 'external_id'...",Take and store data backups from end user syst...,Data Backup,course-of-action--3efe43d1-6f3f-4fcb-ab39-4a73...,course-of-action,2020-03-31T13:11:28.201Z,2019-07-19T14:33:33.543Z,1.1,NaN,NaN,NaN


### Get All Enterprise Groups ONLY (TAXII)

In [77]:
print("Number of Groups in Enterprise ATT&CK")
groups = lift.get_enterprise_groups()
print(len(groups))

Number of Groups in Enterprise ATT&CK
124


### Get All PRE Groups ONLY (TAXII)

In [78]:
print("Number of Groups in PRE-ATT&CK")
groups = lift.get_pre_groups()
print(len(groups))

Number of Groups in PRE-ATT&CK
7


/usr/local/lib/python3.8/site-packages/attackcti/attack_api.py:473: UserWarning: PRE ATT&CK is deprecated. It will be removed in future versions. Consider adjusting your application
  warnings.warn("PRE ATT&CK is deprecated. It will be removed in future versions. Consider adjusting your application")


In [79]:
groups_list = []
for t in groups:
    groups_list.append(json.loads(t.serialize()))
df = pandas.json_normalize(groups_list)
df[0:4]

,created_by_ref,object_marking_refs,external_references,name,description,type,id,aliases,modified,created,x_mitre_version,x_mitre_contributors
0,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,[{'url': 'https://attack.mitre.org/groups/G008...,TEMP.Veles,[TEMP.Veles](https://attack.mitre.org/groups/G...,intrusion-set,intrusion-set--9538b1a4-4120-4e2d-bf59-3b11fca...,"[TEMP.Veles, XENOTIME]",2021-02-09T14:34:04.242Z,2019-04-16T15:14:38.533Z,1.2,NaN
1,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'source_name': 'mitre-attack', 'url': 'https...",APT17,[APT17](https://attack.mitre.org/groups/G0025)...,intrusion-set,intrusion-set--090242d7-73fc-4738-af68-20162f7...,"[APT17, Deputy Dog]",2020-10-13T22:33:14.018Z,2017-05-31T21:31:57.307Z,1.1,NaN
2,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'source_name': 'mitre-attack', 'url': 'https...",APT16,[APT16](https://attack.mitre.org/groups/G0023)...,intrusion-set,intrusion-set--d6e88e18-81e8-4709-82d8-973095d...,[APT16],2020-10-12T19:54:58.537Z,2017-05-31T21:31:56.270Z,1.1,NaN
3,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'source_name': 'mitre-attack', 'url': 'https...",Night Dragon,[Night Dragon](https://attack.mitre.org/groups...,intrusion-set,intrusion-set--23b6a0f5-fa95-46f9-a6f3-4549c5e...,[Night Dragon],2020-10-15T00:54:00.656Z,2017-05-31T21:31:51.643Z,1.3,NaN


### Get All Mobile Groups ONLY (TAXII)

In [80]:
print("Number of Groups in Mobile ATT&CK")
groups = lift.get_mobile_groups()
print(len(groups))

Number of Groups in Mobile ATT&CK
5


In [81]:
groups_list = []
for t in groups:
    groups_list.append(json.loads(t.serialize()))
df = pandas.json_normalize(groups_list)
df[0:4]

,created,modified,aliases,created_by_ref,object_marking_refs,external_references,description,name,type,id,x_mitre_version,x_mitre_contributors
0,2020-06-25T17:16:39.168Z,2021-04-26T14:37:33.234Z,"[Windshift, Bahamut]",identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'external_id': 'G0112', 'source_name': 'mitr...",[Windshift](https://attack.mitre.org/groups/G0...,Windshift,intrusion-set,intrusion-set--afec6dc3-a18e-4b62-b1a4-5510e1a...,1.1,NaN
1,2020-01-27T16:55:39.688Z,2020-03-26T20:58:44.722Z,[Bouncing Golf],identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'external_id': 'G0097', 'source_name': 'mitr...",[Bouncing Golf](https://attack.mitre.org/group...,Bouncing Golf,intrusion-set,intrusion-set--049cef3b-22d5-4be6-b50c-9839c7a...,1.0,NaN
2,2018-10-17T00:14:20.652Z,2020-06-03T20:22:40.401Z,[Dark Caracal],identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'source_name': 'mitre-attack', 'url': 'https...",[Dark Caracal](https://attack.mitre.org/groups...,Dark Caracal,intrusion-set,intrusion-set--8a831aaa-f3e0-47a3-bed8-a9ced74...,1.2,NaN
3,2017-05-31T21:32:04.588Z,2021-04-13T13:13:09.511Z,"[Sandworm Team, ELECTRUM, Telebots, IRON VIKIN...",identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'source_name': 'mitre-attack', 'url': 'https...",[Sandworm Team](https://attack.mitre.org/group...,Sandworm Team,intrusion-set,intrusion-set--381fcf73-60f6-4ab2-9991-6af3cbc...,2.0,NaN


### Get All Groups (TAXII)

* This function gathers all groups defined in each Matrix (Enterprise, PRE & Mobile) and returns the unique ones
  * This is because groups can be repeated across matrices

In [82]:
print("Number of Groups in ATT&CK")
groups = lift.get_groups()
print(len(groups))

Number of Groups in ATT&CK
127


In [83]:
groups_list = []
for t in groups:
    groups_list.append(json.loads(t.serialize()))
df = pandas.json_normalize(groups_list)
df[0:4]

,created_by_ref,object_marking_refs,external_references,description,name,type,id,aliases,modified,created,x_mitre_version,x_mitre_contributors,revoked
0,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'external_id': 'G0130', 'source_name': 'mitr...",[Ajax Security Team](https://attack.mitre.org/...,Ajax Security Team,intrusion-set,intrusion-set--fa19de15-6169-428d-9cd6-3ca3d56...,"[Ajax Security Team, Operation Woolen-Goldfish...",2021-04-22T20:13:14.377Z,2021-04-14T13:17:43.941Z,1.0,NaN,NaN
1,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'external_id': 'G0129', 'source_name': 'mitr...",[Mustang Panda](https://attack.mitre.org/group...,Mustang Panda,intrusion-set,intrusion-set--420ac20b-f2b9-42b8-aa1a-6d4b728...,"[Mustang Panda, TA416, RedDelta, BRONZE PRESID...",2021-04-25T21:15:45.895Z,2021-04-12T15:56:28.861Z,1.0,"[Kyaw Pyiyt Htet, @KyawPyiytHtet]",NaN
2,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'external_id': 'G0128', 'source_name': 'mitr...",[ZIRCONIUM](https://attack.mitre.org/groups/G0...,ZIRCONIUM,intrusion-set,intrusion-set--4283ae19-69c7-4347-a35e-b56f08e...,"[ZIRCONIUM, APT31]",2021-04-20T21:00:44.930Z,2021-03-24T15:48:17.731Z,1.0,NaN,NaN
3,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'external_id': 'G0127', 'source_name': 'mitr...",[TA551](https://attack.mitre.org/groups/G0127)...,TA551,intrusion-set,intrusion-set--94873029-f950-4268-9cfd-5032e15...,"[TA551, GOLD CABIN, Shathak]",2021-03-25T15:33:58.444Z,2021-03-19T21:04:00.692Z,1.0,"[Shuhei Sasada, Cyber Defense Institute, Inc, ...",NaN


### Get All Enterprise & Mobile Software (Malware & Tools) (TAXII)

In [84]:
print("Number of Software in ATT&CK")
software = lift.get_software()
print(len(software))

Number of Software in ATT&CK
596


### Get All Enterprise Relationships ONLY (TAXII)

In [85]:
software_list = []
for t in software:
    software_list.append(json.loads(t.serialize()))
df = pandas.json_normalize(software_list)
df[0:4]

,created,modified,labels,type,id,name,description,created_by_ref,object_marking_refs,external_references,x_mitre_platforms,x_mitre_aliases,x_mitre_version,x_mitre_contributors,x_mitre_old_attack_id,revoked
0,2021-03-19T13:11:50.666Z,2021-04-26T22:35:19.315Z,[tool],tool,tool--80c815bb-b24a-4b9c-9d73-ff4c075a278d,Out1,[Out1](https://attack.mitre.org/software/S0594...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'external_id': 'S0594', 'source_name': 'mitr...",[Windows],[Out1],1.0,NaN,NaN,NaN
1,2021-03-18T14:57:34.628Z,2021-04-25T23:30:38.375Z,[tool],tool,tool--03c6e0ea-96d3-4b23-9afb-05055663cf4b,RemoteUtilities,[RemoteUtilities](https://attack.mitre.org/sof...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'external_id': 'S0592', 'source_name': 'mitr...",[Windows],[RemoteUtilities],1.0,NaN,NaN,NaN
2,2021-03-18T13:39:27.676Z,2021-03-18T14:54:01.053Z,[tool],tool,tool--842976c7-f9c8-41b2-8371-41dc64fbe261,ConnectWise,[ConnectWise](https://attack.mitre.org/softwar...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'external_id': 'S0591', 'source_name': 'mitr...",[Windows],"[ConnectWise, ScreenConnect]",1.0,NaN,NaN,NaN
3,2021-03-17T15:26:20.015Z,2021-04-24T20:45:08.323Z,[tool],tool,tool--b63970b7-ddfb-4aee-97b1-80d335e033a8,NBTscan,[NBTscan](https://attack.mitre.org/software/S0...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'external_id': 'S0590', 'source_name': 'mitr...","[Windows, Linux, macOS]",[NBTscan],1.0,"[Daniyal Naeem, BT Security]",NaN,NaN


In [86]:
print("Number of Relationships in Enterprise ATT&CK")
relationships = lift.get_enterprise_relationships()
print(len(relationships))

Number of Relationships in Enterprise ATT&CK
10986


In [87]:
relations_list = []
for t in relationships:
    relations_list.append(json.loads(t.serialize()))
df = pandas.json_normalize(relations_list)
df[0:4]

,created_by_ref,object_marking_refs,external_references,description,id,type,modified,created,source_ref,relationship_type,target_ref
0,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'source_name': 'CISA AppleJeus Feb 2021', 'u...",[AppleJeus](https://attack.mitre.org/software/...,relationship--fbe555c3-5c7b-44e7-a48f-293bdae9...,relationship,2021-04-27T15:46:45.720Z,2021-04-27T15:46:45.720Z,malware--e2d34c63-6f5a-41f5-86a2-e2380f27f858,uses,attack-pattern--ef67e13e-5598-4adc-bdb2-998225...
1,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'source_name': 'CERT-FR PYSA April 2020', 'u...",[Pysa](https://attack.mitre.org/software/S0583...,relationship--fc0b1fb2-f987-4d9c-8470-c40c5179...,relationship,2021-04-27T03:33:35.204Z,2021-04-27T03:33:35.204Z,malware--a19c1197-9414-46e3-986f-0f609ff4a46b,uses,attack-pattern--d63a3fb8-9452-4e9d-a60a-54be68...
2,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,[{'source_name': 'CheckPoint Volatile Cedar Ma...,[Explosive](https://attack.mitre.org/software...,relationship--00038d0e-7fc7-41c3-9055-edb4d87e...,relationship,2021-04-27T01:56:35.810Z,2021-04-27T01:56:35.810Z,malware--6a21e3a4-5ffe-4581-af9a-6a54c7536f44,uses,attack-pattern--707399d6-ab3e-4963-9315-d9d381...
3,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,[{'source_name': 'CheckPoint Volatile Cedar Ma...,[Explosive](https://attack.mitre.org/software/...,relationship--afcc8ef7-cc5e-4e3b-a430-acafb12f...,relationship,2021-04-27T01:56:35.803Z,2021-04-27T01:56:35.803Z,malware--6a21e3a4-5ffe-4581-af9a-6a54c7536f44,uses,attack-pattern--391d824f-0ef1-47a0-b0ee-c59a75...


### Get All PRE Relationships ONLY (TAXII)

In [88]:
print("Number of Relationships in PRE-ATT&CK")
relationships = lift.get_pre_relationships()
print(len(relationships))

Number of Relationships in PRE-ATT&CK
69


/usr/local/lib/python3.8/site-packages/attackcti/attack_api.py:491: UserWarning: PRE ATT&CK is deprecated. It will be removed in future versions. Consider adjusting your application
  warnings.warn("PRE ATT&CK is deprecated. It will be removed in future versions. Consider adjusting your application")


In [89]:
relations_list = []
for t in relationships:
    relations_list.append(json.loads(t.serialize()))
df = pandas.json_normalize(relations_list)
df[0:4]

,created_by_ref,object_marking_refs,external_references,description,id,type,modified,created,source_ref,relationship_type,target_ref
0,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'source_name': 'Mandiant APT1', 'description...",[APT1](https://attack.mitre.org/groups/G0006) ...,relationship--980656e3-ba60-49ee-9ce8-cbe1a0dc...,relationship,2020-03-25T13:59:27.774Z,2020-03-25T13:59:27.774Z,intrusion-set--6a2e693f-24e5-451a-9f88-b36a108...,uses,attack-pattern--4900fabf-1142-4c1f-92f5-0b590e...
1,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'description': 'Miller, S, et al. (2019, Apr...",[TEMP.Veles](https://attack.mitre.org/groups/G...,relationship--21842707-0f15-43bf-bc42-2bceadf2...,relationship,2019-04-29T18:59:16.596Z,2019-04-24T19:45:44.212Z,intrusion-set--9538b1a4-4120-4e2d-bf59-3b11fca...,uses,attack-pattern--20a66013-8dab-4ca3-a67d-766c84...
2,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'description': 'Miller, S, et al. (2019, Apr...",[TEMP.Veles](https://attack.mitre.org/groups/G...,relationship--2d95ed6f-52e7-4708-af15-9a6c0839...,relationship,2019-04-29T18:59:16.595Z,2019-04-24T19:45:44.205Z,intrusion-set--9538b1a4-4120-4e2d-bf59-3b11fca...,uses,attack-pattern--795c1a92-3a26-453e-b99a-6a566a...
3,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,NaN,NaN,relationship--83379e43-4bc5-4c49-b0b3-f41161e8...,relationship,2019-02-19T18:56:56.770Z,2019-02-19T18:56:56.770Z,attack-pattern--e5164428-03ca-4336-a9a7-4d9ea1...,related-to,attack-pattern--03f4a766-7a21-4b5e-9ccf-e0cf42...


### Get All Mobile Relationships ONLY (TAXII)

In [90]:
print("Number of Relationships in Mobile ATT&CK")
relationships = lift.get_mobile_relationships()
print(len(relationships))

Number of Relationships in Mobile ATT&CK
1050


In [91]:
relations_list = []
for t in relationships:
    relations_list.append(json.loads(t.serialize()))
df = pandas.json_normalize(relations_list)
df[0:4]

,created_by_ref,object_marking_refs,external_references,description,id,type,modified,created,source_ref,relationship_type,target_ref
0,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'source_name': 'CitizenLab Circles', 'url': ...",[Circles](https://attack.mitre.org/software/S0...,relationship--718a612e-50c5-40ab-9081-b88cefea...,relationship,2021-04-26T15:33:55.905Z,2021-04-26T15:33:55.905Z,malware--c6a07c89-a24c-4c7e-9e3e-6153cc595e24,uses,attack-pattern--52651225-0b3a-482d-aa7e-10618f...
1,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'source_name': 'CitizenLab Circles', 'url': ...",[Circles](https://attack.mitre.org/software/S0...,relationship--9de24ec5-63bc-4520-9f81-d1b93a31...,relationship,2021-04-26T15:33:55.897Z,2021-04-26T15:33:55.897Z,malware--c6a07c89-a24c-4c7e-9e3e-6153cc595e24,uses,attack-pattern--fb3fa94a-3aee-4ab0-b7e7-abdf0a...
2,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'source_name': 'Lookout Uyghur Campaign', 'u...",[DoubleAgent](https://attack.mitre.org/softwar...,relationship--64ddcf35-dbf0-4b9f-bf07-1e0bde8b...,relationship,2021-04-19T17:05:42.574Z,2021-04-19T17:05:42.574Z,malware--3d6c4389-3489-40a3-beda-c56e650b6f68,uses,attack-pattern--e1c912a9-e305-434b-9172-8a6ce3...
3,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'source_name': 'Lookout Uyghur Campaign', 'u...",[SilkBean](https://attack.mitre.org/software/...,relationship--2f8b5252-551c-4a0d-8e72-8da40507...,relationship,2021-04-19T14:29:46.530Z,2021-04-19T14:29:46.530Z,malware--ddbe5657-e21e-4a89-8221-2f1362d397ec,uses,attack-pattern--b327a9c0-e709-495c-aa6e-00b042...


### Get All Relationships (TAXII)

In [92]:
print("Number of Relationships in ATT&CK")
relationships = lift.get_relationships()
print(len(relationships))

Number of Relationships in ATT&CK
12617


In [93]:
relations_list = []
for t in relationships:
    relations_list.append(json.loads(t.serialize()))
df = pandas.json_normalize(relations_list)
df[0:4]

,created_by_ref,object_marking_refs,external_references,description,id,type,modified,created,source_ref,relationship_type,target_ref
0,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'source_name': 'CISA AppleJeus Feb 2021', 'u...",[AppleJeus](https://attack.mitre.org/software/...,relationship--fbe555c3-5c7b-44e7-a48f-293bdae9...,relationship,2021-04-27T15:46:45.720Z,2021-04-27T15:46:45.720Z,malware--e2d34c63-6f5a-41f5-86a2-e2380f27f858,uses,attack-pattern--ef67e13e-5598-4adc-bdb2-998225...
1,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,"[{'source_name': 'CERT-FR PYSA April 2020', 'u...",[Pysa](https://attack.mitre.org/software/S0583...,relationship--fc0b1fb2-f987-4d9c-8470-c40c5179...,relationship,2021-04-27T03:33:35.204Z,2021-04-27T03:33:35.204Z,malware--a19c1197-9414-46e3-986f-0f609ff4a46b,uses,attack-pattern--d63a3fb8-9452-4e9d-a60a-54be68...
2,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,[{'source_name': 'CheckPoint Volatile Cedar Ma...,[Explosive](https://attack.mitre.org/software...,relationship--00038d0e-7fc7-41c3-9055-edb4d87e...,relationship,2021-04-27T01:56:35.810Z,2021-04-27T01:56:35.810Z,malware--6a21e3a4-5ffe-4581-af9a-6a54c7536f44,uses,attack-pattern--707399d6-ab3e-4963-9315-d9d381...
3,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[marking-definition--fa42a846-8d90-4e51-bc29-7...,[{'source_name': 'CheckPoint Volatile Cedar Ma...,[Explosive](https://attack.mitre.org/software/...,relationship--afcc8ef7-cc5e-4e3b-a430-acafb12f...,relationship,2021-04-27T01:56:35.803Z,2021-04-27T01:56:35.803Z,malware--6a21e3a4-5ffe-4581-af9a-6a54c7536f44,uses,attack-pattern--391d824f-0ef1-47a0-b0ee-c59a75...
